In [415]:
import pandas as pd

In [416]:
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', index_col='Country/Region')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv', index_col='Country/Region')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv', index_col='Country/Region')

In [417]:
df_confirmed.rename(columns={'Province/State':'Province', df_confirmed.columns[-1]:'Confirmed_Cases'}, inplace=True)
df_deaths.rename(columns={'Province/State':'Province', df_deaths.columns[-1]:'Total_Deaths'}, inplace=True)
df_recovered.rename(columns={'Province/State':'Province', df_recovered.columns[-1]:'Total_Recovered'}, inplace=True)

In [418]:
Confirmed = df_confirmed[['Province', 'Confirmed_Cases']]
Deaths = df_deaths[['Province', 'Total_Deaths']]
Recovered = df_recovered[['Province', 'Total_Recovered']]

In [419]:
filt = pd.isnull(Confirmed.Province)
filt1 = pd.isnull(Deaths.Province)
filt2 = pd.isnull(Recovered.Province)

In [420]:
Confirmed = Confirmed.loc[filt]
Deaths = Deaths.loc[filt1]
Recovered = Recovered.loc[filt2]

In [421]:
Recovered.loc['United Kingdom'] # UK has stopped giving recovered cases data.

Province           NaN
Total_Recovered      0
Name: United Kingdom, dtype: object

In [422]:
new = pd.concat([Confirmed, Deaths, Recovered], axis = 'columns')

In [423]:
new.drop(['Province'], axis = 1, inplace = True)

In [424]:
new.nlargest(10, 'Confirmed_Cases')

,Confirmed_Cases,Total_Deaths,Total_Recovered
US,2114026.0,116127.0,576334
Brazil,888271.0,43959.0,477709
Russia,536484.0,7081.0,284021
India,343091.0,9900.0,180013
United Kingdom,296857.0,41736.0,0
Spain,244109.0,27136.0,150376
Italy,237290.0,34371.0,177010
Peru,232992.0,6860.0,119409
Iran,189876.0,8950.0,150590
France,189670.0,29375.0,69719


In [425]:
new.loc['US']

Confirmed_Cases    2114026.0
Total_Deaths        116127.0
Total_Recovered     576334.0
Name: US, dtype: float64

In [426]:
grpC = df_confirmed.groupby(['Country/Region'])
chinaC = grpC.get_group('China')['Confirmed_Cases'].sum()
canadaC = grpC.get_group('Canada')['Confirmed_Cases'].sum()
australiaC = grpC.get_group('Australia')['Confirmed_Cases'].sum()

In [427]:
grpD = df_deaths.groupby(['Country/Region'])
chinaD = grpD.get_group('China')['Total_Deaths'].sum()
canadaD = grpD.get_group('Canada')['Total_Deaths'].sum()
australiaD = grpD.get_group('Australia')['Total_Deaths'].sum()

In [428]:
grpR = df_recovered.groupby(['Country/Region'])
australiaR = grpR.get_group('Australia')['Total_Recovered'].sum()
canadaR = Recovered.loc['Canada', 'Total_Recovered']
chinaR = grpR.get_group('China')['Total_Recovered'].sum()

In [429]:
Arow = pd.Series(data={'Confirmed_Cases':australiaC, 'Total_Deaths':australiaD, 'Total_Recovered':australiaR}, name= 'Australia')
new = new.append(Arow, ignore_index=False)

In [430]:
Chrow = pd.Series(data={'Confirmed_Cases':chinaC, 'Total_Deaths':chinaD, 'Total_Recovered':chinaR}, name= 'China')
new = new.append(Chrow, ignore_index=False)

In [431]:
new.drop(['Canada'], axis=0, inplace=True)

In [432]:
Carow = pd.Series(data={'Confirmed_Cases':canadaC, 'Total_Deaths':canadaD, 'Total_Recovered':canadaR}, name= 'Canada')
new = new.append(Carow, ignore_index=False)

In [433]:
new.tail()

,Confirmed_Cases,Total_Deaths,Total_Recovered
Tajikistan,5097.0,50.0,3503
Lesotho,4.0,0.0,2
Australia,7347.0,102.0,6856
China,84378.0,4638.0,79489
Canada,100763.0,8228.0,61466


In [434]:
new['Active_Cases'] = new['Confirmed_Cases'] - new['Total_Deaths'] - new['Total_Recovered']

In [435]:
new = new.astype(int)
new.nlargest(20, 'Active_Cases')

,Confirmed_Cases,Total_Deaths,Total_Recovered,Active_Cases
US,2114026,116127,576334,1421565
Brazil,888271,43959,477709,366603
United Kingdom,296857,41736,0,255121
Russia,536484,7081,284021,245382
India,343091,9900,180013,153178
Peru,232992,6860,119409,106723
France,189670,29375,69719,90576
Pakistan,148921,2839,56390,89692
Bangladesh,90619,1209,18731,70679
Spain,244109,27136,150376,66597


In [436]:
new.to_json('Covid19JHU.json', indent=2)